<a href="https://colab.research.google.com/github/notpromising/Python-Assignments/blob/main/Expressionss_to_Optimization_and_Machine_Learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Before you turn this problem in, make sure everything runs as expected. First, **restart the kernel** (in the menubar, select Kernel$\rightarrow$Restart) and then **run all cells** (in the menubar, select Cell$\rightarrow$Run All).

Make sure you fill in any place that says `YOUR CODE HERE` or "YOUR ANSWER HERE", as well as your name and collaborators below:

In [ ]:
NAME = "Parth Mahale"
COLLABORATORS = ""

---

# Assignment 9: From Expressions to Optimization and Machine Learning

_This notebook is a derivative work of ["From Expressions to Optimization and Machine Learning"](https://colab.research.google.com/drive/1br-B3hlv-C6mO9FkXSjV3hcULlIPvydE) by [Luca de Alfaro](https://sites.google.com/a/ucsc.edu/luca/home), which is licensed under [CC BY-NC-ND 4.0](https://creativecommons.org/licenses/by-nc-nd/4.0/), distributed by permission of the original author.  This notebook is licensed under [CC BY-NC-ND 4.0](https://creativecommons.org/licenses/by-nc-nd/4.0/) by [Lindsey Kuper](https://users.soe.ucsc.edu/~lkuper/)._

# Instructions

## Notebook Format

This is a homework notebook.  It consists of various types of cells: 

* Text cells: you should read them!
* Code cells: you should run them, as they may set up the problems that you are asked to solve.
* **Solution** cells: These are cells where you should enter a solution.  You will see a marker in these cells that indicates where your work should be inserted:  

```
    # YOUR CODE HERE
```    

* Test cells: These cells contain some tests, and are worth some points.  You should run the cells as a way to debug your code, and to see if you understood the question, and whether the output of your code is produced in the correct format.

**When we grade your notebook, we will run many additional tests in addition to the ones you see here.  (Otherwise, you'd be able to get full credit by hard-coding the desired output!)**

## Working on Your Notebook

To work on your notebook, we recommend using Colab.  Working in Colab has many benefits:

  * You don't have to maintain a working Python environment on your own machine; you can work from any machine that has an internet connection.
  * Colab preserves the revision history, which is useful for many reasons.
  * Your work is automatically saved in Google Drive.
  
In Colab, go to "File > Save a copy in Drive..." and then you'll have your own copy to work on.

## Submitting Your Notebook

Before you turn your finished notebook in, it's a good idea to sure everything runs as expected. First, **factory reset the runtime** in Colab (go to "Runtime > Factory reset runtime").  Then, **run all the cells** (go to "Runtime > Run all") in Colab.

(A word of caution: when you use "Run all", Colab tries to run cells in parallel, which can get you into trouble if later cells run before earlier ones on which they depend.  So, you might instead want to run all the cells sequentially yourself after resetting the runtime.  You can start at the top of the notebook, hold down the Shift key, and press Enter repeatedly to run one cell after another in order.)

Submit your work as follows: 

  * Download the notebook from Colab, clicking on "File > Download .ipynb".
  * Upload the resulting file to [this Google form](https://docs.google.com/forms/d/e/1FAIpQLScYTOpH2ZtlHFxWvLmj2AepJBKwTEOY-nLC8rddHHJx08br2g/viewform?usp=sf_link).
  * **Deadline: 9am Friday, February 28.**

You can submit multiple times, and the last submission before the deadline will be used to assign you a grade.

Make sure you fill in any place that says `YOUR CODE HERE` or "YOUR ANSWER HERE", as well as your name and the names of anyone you collaborated with in the below cells.  For collaborators, list anyone who (for example) discussed the general techniques involved with you, or pointed you to useful Python documentation -- collaboration of that kind is encouraged.  Remember that sharing code or using others' code is not allowed.

In [ ]:
NAME = "Parth Mahale"
COLLABORATORS = ""

## What Happens Next? 

After you submit your notebook, your instructor at some point will retreat to a secret hideout, put on some good music, and run some mysterious scripts.  These will generate two things: 

  * Your grade, which goes into a spreadsheet. 
  * Feedback, shared to you as a PDF file on Google Drive.  The PDF shows your work, your grade, the tests that passed and those that failed, and any comments left by the instructor and the TAs.

# Testing

Make sure to run the following cell to make sure that the Python testing framework, `nose`, is installed.  (It's required for the rest of the notebook to work.)

In [ ]:
try:
    from nose.tools import assert_equal, assert_almost_equal
    from nose.tools import assert_true, assert_false
    from nose.tools import assert_not_equal, assert_greater_equal
except:
    !pip install nose
    from nose.tools import assert_equal, assert_almost_equal
    from nose.tools import assert_true, assert_false
    from nose.tools import assert_not_equal, assert_greater_equal

## From Expressions to Optimization and Machine Learning

In this assignment, we will use the machine learning framework we developed [during lecture](https://colab.research.google.com/drive/12gLy5eRVXxj0ur7p3ylnbc6o5izh_dye) to train some models and make some predictions.  The next several cells set up the code we'll be using.

To start with, here is our `Expr` class and its subclasses for each expression type (variables (`V`), `Plus`, `Minus`, and so on).  There is nothing new here; this code is exactly as it was in assignment 8.  If you want to understand it better, though, you could always review the ["Expressions as Classes" lecture notebook](https://colab.research.google.com/drive/1YWWaTCloFfU4fROR1v9K6W4gNgW2o0vb).

In [ ]:
class Expr:
    """Abstract class representing expressions"""
    
    def __init__(self, *args):
        self.children = list(args)
        self.child_values = None

    def eval(self, env=None):
        """Evaluates the value of the expression with respect to a given 
        environment."""
        # First, we evaluate the children.
        # This is done here using a list comprehension,
        # but we also could have written a for loop.
        child_values = [c.eval(env=env) if isinstance(c, Expr) else c
                        for c in self.children]

        # Then, we evaluate the expression itself.
        if any([isinstance(v, Expr) for v in child_values]):
            # Symbolic result.
            return self.__class__(*child_values)
        else:
            # Concrete result.
            return self.op(*child_values)

    def op(self, *args):
        """The op method computes the value of the expression, given the
        numerical value of its subexpressions.  It is not implemented in 
        Expr, but rather, each subclass of Expr should provide its 
        implementation."""
        raise NotImplementedError()

    def __repr__(self):
        """Represents the expression as the name of the class,
        followed by all the children in parentheses."""
        return "%s(%s)" % (self.__class__.__name__,
                        ', '.join(repr(c) for c in self.children))

    # Expression constructors
    def __add__(self, other):
        return Plus(self, other)

    def __radd__(self, other):
        return Plus(self, other)

    def __sub__(self, other):
        return Minus(self, other)

    def __rsub__(self, other):
        return Minus(other, self)

    def __mul__(self, other):
        return Multiply(self, other)

    def __rmul__(self, other):
        return Multiply(other, self)

    def __truediv__(self, other):
        return Divide(self, other)

    def __rtruediv__(self, other):
        return Divide(other, self)

    def __pow__(self, other):
        return Power(self, other)

    def __rpow__(self, other):
        return Power(other, self)

    def __neg__(self):
        return Negative(self)
    
class V(Expr):
    """Variable."""
    
    def __init__(self, *args):
        """Variables must be of type string."""
        assert len(args) == 1
        assert isinstance(args[0], str)
        super().__init__(*args)

    def eval(self, env=None):
        """If the variable is in the environment, returns the
        value of the variable; otherwise, returns the expression."""
        if env is not None and self.children[0] in env:
            return env[self.children[0]]
        else:
            return self
        
class Plus(Expr):
    def op(self, x, y):
        return x + y

class Minus(Expr):
    def op(self, x, y):
        return x - y

class Multiply(Expr):
    def op(self, x, y):
        return x * y

class Divide(Expr):
    def op(self, x, y):
        return x / y

class Power(Expr):
    def op(self, x, y):
        return x ** y

class Negative(Expr):
    def op(self, x):
        return -x

To make sure everything is in order, let's try constructing and evaluating some expressions:

In [ ]:
e = (V('x') + V('y')) * (2 + V('x'))
# Evaluate the above expression in an environment where x is bound to 4 and y is bound to 3.
e.eval({'x': 4, 'y': 3})

42

In [ ]:
e = Power(V('x'), 2) + V('y') -1
# Evaluate the above expression in an environment where x is bound to 6 and y is bound to 7.
e.eval({'x': 6, 'y': 7})

42

Next, here's the code for computing derivatives of various expression types.  There's nothing new here, either; we've seen it all before during lecture or in assignment 8.

In [ ]:
def expr_derivative(self, var):
    """Computes the derivative of the expression with respect to var."""
    partials = [(c.derivative(var) if isinstance(c, Expr) else 0)
                for c in self.children]
    return self.op_derivative(var, partials).eval()

def expr_op_derivative(self, var, partials):
    raise NotImplementedError()
    
# Extends the existing Expr class with the newly defined methods.
Expr.derivative = expr_derivative
Expr.op_derivative = expr_op_derivative

def variable_derivative(self, var):
    return 1 if self.children[0] == var else 0

# Extends the existing V class with the newly defined method.
V.derivative = variable_derivative

def plus_op_derivative(self, var, partials):
    return Plus(partials[0], partials[1])

# Extends the existing Plus class...you get the idea.
Plus.op_derivative = plus_op_derivative

def minus_op_derivative(self, var, partials):
    """Implements derivative for Minus expressions."""    
    return Minus(partials[0], partials[1])

Minus.op_derivative = minus_op_derivative

def negative_op_derivative(self, var, partials):
    return Negative(partials[0])

Negative.op_derivative = negative_op_derivative

def multiply_op_derivative(self, var, partials):
    return Plus(
        Multiply(partials[0], self.children[1]),
        Multiply(partials[1], self.children[0])
    )

Multiply.op_derivative = multiply_op_derivative

We will briefly make sure that this works.

In [ ]:
e = 5 + V('y')
e.derivative('y')

1

In [ ]:
e = 4 * V('x')
e.derivative('x')

Plus(Multiply(0, V('x')), 4)

Lastly, here are the `train_model` function and the `plot_points_and_model` function that we developed [during lecture](https://colab.research.google.com/drive/12gLy5eRVXxj0ur7p3ylnbc6o5izh_dye).  We will use these for training and visualizing models.

In [ ]:
def train_model(loss, training_examples, params, var_x, var_y, delta=0.0001, num_iterations=10000):
    env = {param:0 for param in params}
    for iteration_idx in range(num_iterations):        
        gradient = {param:0 for param in params} # Set all gradients to 0
        total_loss = 0.
        for x_val, y_val in training_examples:
            env[var_x] = x_val
            env[var_y] = y_val
            for param in params:
                gradient[param] += loss.derivative(param).eval(env=env)
            total_loss += loss.eval(env=env)
        if (iteration_idx + 1) % 100 == 0:
            print("Loss:", total_loss)
            print("Params: ", {param:env[param] for param in params})
            pass
        for param in params:
            env[param] = env[param] - (delta * gradient[param])
    return total_loss, {param:env[param] for param in params}

import matplotlib
import matplotlib.pyplot as plt

matplotlib.rcParams['figure.figsize'] = (8.0, 8.0)
matplotlib_params = {'legend.fontsize': 'large',
              'axes.labelsize': 'large',
              'axes.titlesize':'large',
              'xtick.labelsize':'large',
              'ytick.labelsize':'large'}
matplotlib.rcParams.update(matplotlib_params)

import numpy as np

def plot_points_and_model(points, var_x, model, params, xlabel='x', ylabel='y'):
    env = params.copy()
    fig, ax = plt.subplots()
    xs, ys = zip(*points)
    ax.plot(xs, ys, 'r+')
    x_min, x_max = np.min(xs), np.max(xs)
    step = (x_max - x_min) / 100
    x_list = list(np.arange(x_min, x_max + step, step))
    y_list = []
    for x in x_list:
        env[var_x] = x
        y_list.append(model.eval(env=env))
    ax.plot(x_list, y_list)
    plt.xlabel(xlabel)
    plt.ylabel(ylabel)
    plt.show()

### Problem 1: Setting up a (slightly) fancier model to predict rental prices

During lecture, we considered the problem of predicting prices of Santa Cruz rental units.  `rental_prices` is a list of training examples, where the first element in each tuple is the number of bedrooms in the rental unit and the second element is the monthly rent.

In [ ]:
rental_prices = [
    (2, 2700),
    (0, 2045),
    (2, 2845),
    (2, 2950),
    (1, 2100),
    (2, 3100),
    (2, 2600),
    (0, 2200),
    (2, 2650),
    (2, 2200),
    (2, 2100),
    (3, 3675),
    (3, 3800),
    (2, 4300),
    (2, 2500),
    (1, 2000),
    (0, 1300),
    (1, 2500),
    (1, 1800),
    (0, 1250),
    (3, 3850),
    (3, 4000)
]

We trained a model that learns the best line through the data.  The resulting model predicted that a 4-bedroom rental would cost around \$4300 a month.  But what if the best fit for the data isn't a line at all?

For this problem, you will use the same training data to train a model shaped like a _parabolic curve_ instead of a line.  In other words, instead of training a model represented by the equation

$$
\hat{y} = ax + b \;,
$$

you will train a model represented by the equation

$$
\hat{y} = ax^2 + bx + c \;.
$$

So, this time, instead of having two parameters to learn ($a$ and $b$), you now have three parameters to learn ($a$, $b$, and $c$).  The framework that we've already developed is perfectly capable of handling this, and you can continue to use the squared error loss function, $(\hat{y} - y)^2$, as the function to minimize during training.

In the following cell, define the variables and expressions needed to set up the problem.  **If you're not sure what to do here, look at the code we used to set up the $\hat{y} = ax + b$ problem during lecture.  It's in the section ["Back to machine learning: setting up the problem"](https://colab.research.google.com/drive/12gLy5eRVXxj0ur7p3ylnbc6o5izh_dye#scrollTo=Q2Ah2C5VWLsc) of the lecture notebook.  You need to do something analogous to what was done there, but now using the model $\hat{y} = ax^2 + bx + c$.**

In [ ]:
# In this cell, you need to define the following variables:
#   * Variables `a`, `b`, and `c`, representing the parameters that we want to train.
#   * Variables `x` and `y`, representing the pairs in the set of training examples.
#   * A variable `y_hat` representing the model we will train.
#   * A variable `loss` representing the loss function.

# Variables `a`, `b`, and `c`, representing the parameters that we want to train
a = V('a')
b = V('b')
c = V('c')


# Variables `x` and `y`, representing the pairs in the set of training examples
x = V('x')
y = V('y')

# A variable `y_hat` representing the model we will train.
y_hat = (a * x*x) + b*(x)+c


# A variable `loss` representing the loss function.
loss = (y_hat - y) * (y_hat - y)


In [ ]:
# Tests for problem setup.
# If you defined the above expressions correctly, the following tests should pass.
assert_equal(y_hat.eval(env={'a': 0, 'b': 0, 'c': 0, 'x': 0, 'y': 0}), 0)
assert_equal(y_hat.eval(env={'a': 8, 'b': 4, 'c': 2, 'x': 2, 'y': 2}), 42)
assert_equal(loss.eval(env={'a': 0, 'b': 0, 'c': 0, 'x': 0, 'y': 0}), 0)
assert_equal(loss.eval(env={'a': 0, 'b': 1, 'c': 0, 'x': 2, 'y': 2}), 0)


In [ ]:
# More tests for problem setup.
# If you defined the above expressions correctly, the following tests should pass.
assert_equal(loss.eval(env={'a': 4, 'b': 5, 'c': 17, 'x': 0, 'y': 17}), 0)
assert_equal(loss.eval(env={'a': 0, 'b': 2, 'c': 3, 'x': 1, 'y': 5}), 0)


### Problem 2: Training our fancier model

For this problem, you will call the `train_model` function to train your model using the `rental_prices` training data, and assign the result of calling `train_model` to the variables `total_loss` and `params`.  The `train_model` function will perform gradient descent and learn the settings of the parameters that minimize the loss.

For reference, here was how we called the `train_model` function during lecture when we trained the $\hat{y} = ax + b$ model (see the ["Training a model that minimizes the loss"](https://colab.research.google.com/drive/12gLy5eRVXxj0ur7p3ylnbc6o5izh_dye#scrollTo=S1ZKJxpM6jDn&line=10&uniqifier=1) section of the lecture notebook):

```
total_loss, params = train_model(loss, rental_prices, ['a', 'b'], 'x', 'y')
```

You need to do the analogous thing, but for the model $\hat{y} = ax^2 + bx + c$.

**Important: pass the argument `num_iterations=30000` to `train_model`.  Otherwise, training will end too soon, before the loss has been reasonably minimized.**

Do not pass a `delta` argument to `train_model`; the `train_model` function should just use the already-specified default value for `delta`.

After you've set up the call to `train_model`, you can run the following cell!  **Warning: if you've set things up correctly, `train_model` will take a few minutes to run.  Now might be a good time to go get some coffee.**

In [ ]:
# In this cell, you need to call the train_model function,
# and assign the result to the variables total_loss and params.
# Your code should look like:
#
# total_loss, params = train_model(...)
#
# where ... is replaced by the appropriate arguments.

total_loss, params = train_model(loss, rental_prices, ['a', 'b','c'], 'x', 'y',num_iterations=30000)
total_loss, params

print("Total loss:", total_loss, "Params:", params)

Loss: 20598043.360091895
Params:  {'a': 372.0396234129675, 'b': 282.0338529584453, 'c': 326.5276202458154}
Loss: 15249495.998575501
Params:  {'a': 305.2166871276627, 'b': 362.1259509758519, 'c': 532.6014502886093}
Loss: 11724974.77571642
Params:  {'a': 251.87331483275295, 'b': 424.28615724191775, 'c': 701.256527245521}
Loss: 9398991.305553805
Params:  {'a': 209.43063476662076, 'b': 471.99899722500004, 'c': 839.5388876296952}
Loss: 7860754.046884042
Params:  {'a': 175.79764874022965, 'b': 508.084903293072, 'c': 953.1595592995848}
Loss: 6840461.954652707
Params:  {'a': 149.28061655665888, 'b': 534.8269964137831, 'c': 1046.7480129347123}
Loss: 6160902.577095005
Params:  {'a': 128.50769820353372, 'b': 554.0730559437345, 'c': 1124.0572234502051}
Loss: 5705667.174005716
Params:  {'a': 112.36788754216609, 'b': 567.3181496319694, 'c': 1188.1298449585765}
Loss: 5398277.968906235
Params:  {'a': 99.96152724511033, 'b': 575.7715915032027, 'c': 1241.4328736595012}
Loss: 5188481.22873268
Params:  {'

If you set up the call to `train_model` correctly, after it's finished running, the `total_loss` variable should be a number considerably less than `4941134.618681211`, which was the total loss for the $\hat{y} = ax + b$ model that we trained during lecture.  The `params` variable should be a dictionary containing the trained values of the parameters to your model.

In [ ]:
# Basic tests of the trained model.
# If you called train_model with the correct arguments,
# and assigned the result to the correct variables,
# the following tests should pass.

assert_greater_equal(4941134.618681211, total_loss)
assert_equal(len(params), 3)

In [ ]:
# More specific tests of the trained model.
# If you called train_model with the correct arguments,
# and assigned the result to the correct variables,
# the following tests should pass.

# Note: If you're not ending up with these parameter values,
# don't forget to use 30,000 training iterations.

assert_greater_equal(params['a'], 160)
assert_greater_equal(161, params['a'])

assert_greater_equal(params['b'], 227)
assert_greater_equal(228, params['b'])

assert_greater_equal(params['c'], 1701.5)
assert_greater_equal(1702.5, params['c'])

### Problem 3: Making predictions with our fancier model

Let's visualize the model we just trained using the following call to `plot_points_and_model`.  If you solved the previous two problems, you should see a nice curve:

In [ ]:
plot_points_and_model(rental_prices, 'x', y_hat, params, xlabel="Number of bedrooms", ylabel="Monthly rent")

For this problem, you will use the trained model `y_hat` to predict the rent for 4-bedroom rental units in Santa Cruz.  There are multiple ways to do this; here is what we suggest:

  * Make a copy of the `params` dictionary.  (You can copy a dictionary using the `.copy()` method.)
  * Add a new entry to the dictionary that associates `'x'` to a value.  For instance, if you want to predict the rent for a 2-bedroom rental unit, `'x'` should be associated with `2`.  The dictionary should now contain entries for all of the learned parameters, plus `'x'`.
  * Call the `eval` method on `y_hat`, your model, passing your dictionary as the `env` argument.
  
You can do the above in no more than 3 lines of code.  If you're not sure what to do, the [lecture notebook](https://colab.research.google.com/drive/12gLy5eRVXxj0ur7p3ylnbc6o5izh_dye) has an example near the end.
  
The result of evaluating `y_hat` with an environment as described above will give you the predicted monthly rent of a unit with the specified number of bedrooms.

**In the below cell, assign the predicted rent for 4-bedroom rental units to the variable `predicted_4_bedroom_rent`.**

In [ ]:
# In this cell, use your trained model, `y_hat`, to predict the rent for a 4-bedroom rental unit
# and assign the resulting value to the variable `predicted_4_bedroom_rent`.

env = params.copy()
env['x'] = 4
predicted_4_bedroom_rent=y_hat.eval(env=env)

print("Predicted rent:", predicted_4_bedroom_rent)

In [ ]:
assert_greater_equal(predicted_4_bedroom_rent, 5100)


Do you think the $\hat{y} = ax^2 + bx + c$ model does a better job of predicting the price for 4-bedroom rentals than the $\hat{y} = ax + b$ model that we saw in lecture did?  Feel free to compare the predictions with [real data](https://www.zillow.com/santa-cruz-ca/rentals/4-4_beds/).